<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Advanced Data Science<br/>Asignatura Reinforcement Learning</center>

# Challenge S1: Práctica con Gym

In [ ]:
# !pip install gym
# !pip install pyglet


In [1]:
import gym
import matplotlib.pyplot as plt
%matplotlib inline
from s1_utils import env_wrapper
# from IPython import display
# import os
# os.environ['DISPLAY'] = ':1'

El principal **objetivo** de esta práctica es sentirnos cómodos utilizando el framework Gym para manejar entornos de problemas de Reinforcement Learning.

El el worksheet ya hemos visto el entorno MountainCar de OpenAI Gym. Ahora, buscándote la vida, debes obtener una lista llamada env_names con todos los entornos que contiene la librería e imprimirla a continuación.

In [2]:
from gym import envs

env_names = [env.id for env in envs.registry.all()] ## YOUR CODE HERE ##
for name in sorted(env_names):
    print(name)

Acrobot-v1
Ant-v2
Ant-v3
BipedalWalker-v3
BipedalWalkerHardcore-v3
Blackjack-v1
CarRacing-v0
CartPole-v0
CartPole-v1
CliffWalking-v0
CubeCrash-v0
CubeCrashScreenBecomesBlack-v0
CubeCrashSparse-v0
FetchPickAndPlace-v1
FetchPickAndPlaceDense-v1
FetchPush-v1
FetchPushDense-v1
FetchReach-v1
FetchReachDense-v1
FetchSlide-v1
FetchSlideDense-v1
FrozenLake-v1
FrozenLake8x8-v1
HalfCheetah-v2
HalfCheetah-v3
HandManipulateBlock-v0
HandManipulateBlockDense-v0
HandManipulateBlockFull-v0
HandManipulateBlockFullDense-v0
HandManipulateBlockRotateParallel-v0
HandManipulateBlockRotateParallelDense-v0
HandManipulateBlockRotateParallelTouchSensors-v0
HandManipulateBlockRotateParallelTouchSensors-v1
HandManipulateBlockRotateParallelTouchSensorsDense-v0
HandManipulateBlockRotateParallelTouchSensorsDense-v1
HandManipulateBlockRotateXYZ-v0
HandManipulateBlockRotateXYZDense-v0
HandManipulateBlockRotateXYZTouchSensors-v0
HandManipulateBlockRotateXYZTouchSensors-v1
HandManipulateBlockRotateXYZTouchSensorsDense-v

A continuación escribe una función que visualice el entorno que más te guste de *env_names* (argumento de la función), ejecute 250 steps y si consigue finalizarlo imprima un mesaje de enhorabuna con el número de steps que ha necesitado para llegar al final y la reconpensa acumulada final.

In [3]:
def render_env(myenv):
    ## YOUR CODE HERE ##
    steps = 250
    observation = myenv.reset()
    # al empezar, tomamos la observación anterior como la primera
    previous_observation = observation

    # Ejecutamos 250 pasos en el entorno
    try:
        for step in range(steps):
        # Visualizamos el juego de MountainCar
            myenv.render()

            # Tomamos una acción aleatoriamente
            action = myenv.action_space.sample()

            # Ejecutamos la acción
            observation, reward, done, info = myenv.step(action) 
            
             ## Si se ha llegado al objetivo done = True y por lo tanto se corta el juego. Hacemos break paa salir del bucle.
            if done:
                print(f'Enhorabuena! Lo has logrado en {step} pasos')
                break
            
        ## Hemos salido del bucle ejecutanto los pasos máximos establecidos y no lo hemos conseguido.
        else:
            print(f'Lo siento! No has alcanzado el objetivo')
    except Exception as e:
        raise e
    finally:
        myenv.close()

In [4]:
render_env(gym.make("Pendulum-v1").unwrapped)

Lo siento! No has alcanzado el objetivo


## `env_wrapper`

Comenta qué hace cada parte del código de la función en las líneas con `##`. Es decir, completa los comentarios de la función `env_wrapper`, explicando qué hace cada uno de los métodos usados sobre `env`.

In [22]:
def env_wrapper(policy_func, env,steps):
    '''
    Simula el entorno `env` siguiendo la política `policy_func`.
    '''
    ## Resetea el entorno y devuelve la observación inicial
    observation = env.reset()
    # al empezar, tomamos la observación anterior como la primera
    previous_observation = observation

    # Ejecutamos 250 pasos en el entorno
    try:
        for step in range(steps):
            ## Visulizamos en una ventana el estado actual del entorno.
            env.render()

            ## Determina que accion tomar. Esta acción dependerá de la politica que hayamos considerado para el entorno
            ## podría ser aleatoria, determinista y probabilistica
            action = policy_func(observation, step, previous_observation)
#             print(f'wrapper action {action}')
            # actualizamos previous_observation
            previous_observation = observation
            
            ## ejecuta la acción previamnte calculada y nos devuelve una nueva observación (obsrrvation),
            ## una recompensa (reward), si el entorno ha llegado al obetivo (done), e información adicional
            observation, reward, done, info = env.step(action)
           ## Si se ha llegado al objetivo done = True y por lo tanto se corta el juego. Hacemos break paa salir del bucle.
            if done:
                print(f'Enhorabuena! Lo has logrado en {step} pasos')
                break
                
            
         ## Hemos salido del bucle ejecutanto los pasos máximos establecidos y no lo hemos conseguido. 
        else:
            print(f'Lo siento! No has alcanzado el objetivo')
    except Exception as e:
        raise e
    finally:
        env.close()

## Comprendiendo los espacios

In [23]:
# unwrapped evita que tenga un limite temporal de 200
env = gym.make("MountainCar-v0").unwrapped

Ahora, retomaremos el ejemplo del entorno *MountainCar* (https://github.com/openai/gym/wiki/MountainCar-v0) presentado durante el [Worksheet](./s1_worksheet.ipynb). En este, se probó a utilizar una política aleatoria que apenas conseguía mover el coche desde su posición inicial y que no utilizaba ningún dato del estado del entorno para moverse. 

Para solucionar esto, durante el challenge vamos a aplicar distintas formas de observar el entorno y así conseguir optimizarlo. Hemos creado una función, `env_wrapper` que simula la interacción del agente con un entorno dado por `env`, dada una función de política.

Una vez cargado el entorno, vemos que no tiene un estado definido por defecto. Es necesario aplicar el método `.reset()` para que el agente tenga definido un estado.

En este caso, el estado viene dado por un array con dos variables, la posición y la velocidad. Que como podemos observar, empieza cerca de la posición -0.56 y con velocidad nula.

In [24]:
s0 = env.reset()
print(f's0 = {s0}')
print(f'env.state = {env.state}')

s0 = [-0.5611406  0.       ]
env.state = [-0.56114059  0.        ]


A continuación vamos a ver una descripción del espacio de estados y de acciones ¡¡Prueba con otro juego de la lista!!. Observa los resultados. *Nota:* Algunos juegos pueden dar error en el SO Windows

In [25]:
from gym.spaces import *

def print_spaces(space):
    print(space)
    if isinstance(space, Box): # Comprueba si el space es de tipo box, imprime las fronteras
           print ('\n Cota inferior; ', space.low)
           print ('\n Cota superior; ', space.high) 
           

environment = gym.make('MountainCar-v0')
print('Espacio de estados:')
print_spaces(environment.observation_space)
print('Espacio de acciones: ')
print_spaces(environment.action_space)
try:
    print('Descripcion de las acciones. ', environment.unwrapped.get_action_meanings())
except AttributeError:
    pass

Espacio de estados:
Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

 Cota inferior;  [-1.2  -0.07]

 Cota superior;  [0.6  0.07]
Espacio de acciones: 
Discrete(3)


Comprendidos los espacios de estados y acciones, ahora como ejemplo de cómo utilizar `env_wrapper`, vamos a utiliar una polítca que siempre toma la acción de impulsar hacia la izquierda. 

Como puedes observar, esta política no es suficientemente inteligente para poder alcanzar la meta.

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython import display

import gym 
env = gym.make("MountainCar-v0").unwrapped
env.reset()
for _ in range(1000): 
    env.render() 
    env.step(env.action_space.sample()) # take a random action 
env.close()

In [27]:
steps = 300
def dummy_policy(observarion, *args):
    return 0
env_wrapper(dummy_policy, env, steps)

Lo siento! No has alcanzado el objetivo


## Policy determinista

En primer lugar implementa una función determinista que consiga lograr el objetivo de permitir al coche alcanzar la meta superior derecha. Para ello, implementamos una función que recibe la observación actual del entorno y el contador de iteraciones ejecutadas, y devuelve la acción a ejecutar.

Recuerda, las acciones posibles son:
* 0: impulsar a la izquierda
* 1: no hacer nada
* 2: impulsar a la derecha

**Tip:** The optimal policy is to move on the left accumulating inertia and then to push as much as possible to the right.

In [28]:
def policy_determinista(observation, step, *args):
    x, v = observation    
    print('Step: '+str(step),'x:',x,' v:',v)    
    
    # ESCRIBE TU CÓDIGO
    
    action=0 if v < 0 and x < 0 else 2
       
    return action 
    
    

In [29]:
steps = 250
env_wrapper(policy_determinista, env, steps)

Step: 0 x: -0.45117584  v: 0.0
Step: 1 x: -0.45071474  v: 0.00046109132
Step: 2 x: -0.44979593  v: 0.0009188063
Step: 3 x: -0.44842613  v: 0.0013697963
Step: 4 x: -0.44661537  v: 0.0018107684
Step: 5 x: -0.44437686  v: 0.0022385118
Step: 6 x: -0.44172692  v: 0.0026499252
Step: 7 x: -0.43868488  v: 0.0030420418
Step: 8 x: -0.43527284  v: 0.0034120532
Step: 9 x: -0.4315155  v: 0.0037573336
Step: 10 x: -0.42744005  v: 0.0040754597
Step: 11 x: -0.42307582  v: 0.00436423
Step: 12 x: -0.41845414  v: 0.0046216836
Step: 13 x: -0.413608  v: 0.004846111
Step: 14 x: -0.40857196  v: 0.005036069
Step: 15 x: -0.40338156  v: 0.0051903883
Step: 16 x: -0.39807338  v: 0.0053081783
Step: 17 x: -0.39268455  v: 0.0053888313
Step: 18 x: -0.38725254  v: 0.005432022
Step: 19 x: -0.38181484  v: 0.0054377033
Step: 20 x: -0.37640873  v: 0.0054061
Step: 21 x: -0.37107104  v: 0.005337701
Step: 22 x: -0.36583778  v: 0.0052332478
Step: 23 x: -0.36074406  v: 0.0050937207
Step: 24 x: -0.35582373  v: 0.0049203243
Step:

## Policy probabilista

¿Se te ocurre alguna manera de convertir la política anterior a un enfoque probabilístico?

**Ayuda:** Puedes seguir una estrategia tipo $\epsilon$-greedy como se describe [aquí](https://es.wikipedia.org/wiki/Aprendizaje_por_refuerzo#Exploraci%C3%B3n). 
Es decir, puedes convertir la política anterior a una probabilística diciéndole que un porcentaje de los casos realice una acción al azar en lugar de determinista. En este caso, **¿qué pasa si aumentas la probabilidad de realizar acciones al azar?**. Puedes leer una breve introducción sobre este algoritmo y su importancia en Reinforcement Learning en [link](https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870).

Puedes crear un función probabilística que siga la política $\epsilon$-greedy, es decir, según esta política puedes coger una acción aleatoria o la acción determinista correspondiente calculada en el apartado anterior.  **No olvides** incluir los parámetros referentes a épsilon como argumentos en las funciones que correspondan ni contestar a la pregunta. Si tienes que modificar el wrapper anterior no olvides hacerte una **copia** de la función y **renombrarlo**.

**Respuesta:**
* Siguiendo la sugerencia hemos implementado una estrategia e-gredy con descenso del valor de epsilon. Esta estrategía está muy extendida en el ámbito del RL además de ser relativamente sencilla de implementar. 
* Por lo que he podio validar en as pruebas que he realizado contra más alto es epsilon y **más cercano es a 1 la política que con más frecuencia  se eligirá será la Aleatoria** con más probabilidad de no conseguir exito y de ternlo en mayor número de pasos. Por contra contra **más cercano sea al 0 la política más frecuente será la Determinista** y por lo tanto con más probabilidad de conseguir éxito y en menor número de pasos. He realizado dos ejemplos uno con un epsilon=1 y otro con epsilon=0.25 y se demuestra que el de 0,25 que tiene menos probabilidad de tomar acciones al azar llega antes a tener éxito, encontra del inicializado a 1 que si llega al exito es apurando al máxmimo el número de pasos, existiendo ocasiones en las que no llega a éxito. **En conclusión** a mayor probabilidad de tomar acciones al azar mayor riesgo de no alcanzar el objetivo.
* También comentar que he visto distintas estrategias para hacer la degradación del epsilon y me he quedado con **epsilon-=EPSILON_DECAY**, otra que también he podido observar es **epsilon*=(1-EPSILON_DECAY)**. 

In [30]:
import numpy as np # Puedes usar el método random de numpy para elegir una acción aleatoria

def env_wrapper_probabilistico(policy_func, env,steps,myepsilon=1.0):
    '''
    Simula el entorno `env` siguiendo la política `policy_func`.
    '''
    steps = 250 
    EPSILON_MIN = 0.05 ## YOUR CODE HERE ##
    EPSILON_DECAY = (1-EPSILON_MIN) / steps
    epsilon = myepsilon
    
    ## Resetea el entorno y devuelve la observación inicial
    observation = env.reset()
    # al empezar, tomamos la observación anterior como la primera
    previous_observation = observation

    # Ejecutamos 250 pasos en el entorno
    try:
        for step in range(steps):
            ## Visulizamos en una ventana el estado actual del entorno.
            env.render()

            ## Determina que accion tomar. Esta acción dependerá de la politica que hayamos considerado para el entorno
            ## podría ser aleatoria, determinista y probabilistica
            action = policy_func(observation, step, previous_observation,env,epsilon)
#             print(f'wrapper action {action}')
            # actualizamos previous_observation
            previous_observation = observation
            
            ## ejecuta la acción previamnte calculada y nos devuelve una nueva observación (obsrrvation),
            ## una recompensa (reward), si el entorno ha llegado al obetivo (done), e información adicional
            observation, reward, done, info = env.step(action)
            ## Si se ha llegado al objetivo done = True y por lo tanto se corta el juego. Hacemos break paa salir del bucle.
            if done:
                print(f'Enhorabuena! Lo has logrado en {step} pasos')
                break
                
            # Reducimos el epsilon hasta que llege al mínimo. Lo hacemos con ua resta
            if epsilon >= EPSILON_MIN:
                epsilon -= EPSILON_DECAY    
            
        ## Hemos salido del bucle ejecutanto los pasos máximos establecidos y no lo hemos conseguido. 
        else:
            print(f'Lo siento! No has alcanzado el objetivo')
    except Exception as e:
        raise e
    finally:
        env.close()

def policy_probabilista(observation, step, *args):
    
    # YOUR CODE HERE #
    
    #Calculo valor de epsilom
    #calculo numero aletorio
    #ver si es mayor o menor que epsilosn
    # si mayor llamar a policy_determinista.
    #si menor llamar a politica aleatoria env.action_s
    #Obtenemos el modelo que lo hemos pasado en los args provinientte de la función env_wrapper
    
    myenv=args[1]
    myepsilon=args[2]
    p = np.random.uniform(0, 1)
    print(f'Step: {step} Número Aleatorio: {p} Epsilon: {myepsilon}')
   
    if (p < myepsilon):
        action = myenv.action_space.sample()
        print(f'Step: {step} Política Aleatoria - Accion {action}')

    else:
        action = policy_determinista(observation, step)
        print(f'Step: {step} Política Determinista - Accion {action}')

    return action

In [34]:
env_wrapper_probabilistico(policy_probabilista, env,steps,1.0) # O invoca tu wrapper modificado

Step: 0 Número Aleatorio: 0.90117591438294 Epsilon: 1.0
Step: 0 Política Aleatoria - Accion 1
Step: 1 Número Aleatorio: 0.8684455106131134 Epsilon: 0.9962
Step: 1 Política Aleatoria - Accion 2
Step: 2 Número Aleatorio: 0.09321372222083957 Epsilon: 0.9924
Step: 2 Política Aleatoria - Accion 0
Step: 3 Número Aleatorio: 0.16364969136445706 Epsilon: 0.9885999999999999
Step: 3 Política Aleatoria - Accion 0
Step: 4 Número Aleatorio: 0.9548017731134211 Epsilon: 0.9847999999999999
Step: 4 Política Aleatoria - Accion 1
Step: 5 Número Aleatorio: 0.6169686465541068 Epsilon: 0.9809999999999999
Step: 5 Política Aleatoria - Accion 2
Step: 6 Número Aleatorio: 0.6733141926659242 Epsilon: 0.9771999999999998
Step: 6 Política Aleatoria - Accion 0
Step: 7 Número Aleatorio: 0.11132901416862861 Epsilon: 0.9733999999999998
Step: 7 Política Aleatoria - Accion 0
Step: 8 Número Aleatorio: 0.16536635224553176 Epsilon: 0.9695999999999998
Step: 8 Política Aleatoria - Accion 1
Step: 9 Número Aleatorio: 0.5527764601

Step: 72 Número Aleatorio: 0.15331658623267785 Epsilon: 0.7263999999999982
Step: 72 Política Aleatoria - Accion 0
Step: 73 Número Aleatorio: 0.29088843617679994 Epsilon: 0.7225999999999981
Step: 73 Política Aleatoria - Accion 0
Step: 74 Número Aleatorio: 0.45697928516577835 Epsilon: 0.7187999999999981
Step: 74 Política Aleatoria - Accion 0
Step: 75 Número Aleatorio: 0.2065209037444391 Epsilon: 0.7149999999999981
Step: 75 Política Aleatoria - Accion 2
Step: 76 Número Aleatorio: 0.5970816743389906 Epsilon: 0.7111999999999981
Step: 76 Política Aleatoria - Accion 0
Step: 77 Número Aleatorio: 0.11749152833432652 Epsilon: 0.707399999999998
Step: 77 Política Aleatoria - Accion 2
Step: 78 Número Aleatorio: 0.6827873756040193 Epsilon: 0.703599999999998
Step: 78 Política Aleatoria - Accion 0
Step: 79 Número Aleatorio: 0.8062556869153275 Epsilon: 0.699799999999998
Step: 79 x: -0.43218222  v: -0.0033451482
Step: 79 Política Determinista - Accion 0
Step: 80 Número Aleatorio: 0.3445610166162004 Epsi

Step: 146 Número Aleatorio: 0.4173604193995897 Epsilon: 0.44519999999999627
Step: 146 Política Aleatoria - Accion 1
Step: 147 Número Aleatorio: 0.08943963438977942 Epsilon: 0.44139999999999624
Step: 147 Política Aleatoria - Accion 0
Step: 148 Número Aleatorio: 0.8951868070365449 Epsilon: 0.4375999999999962
Step: 148 x: -0.13628705  v: 0.0062537924
Step: 148 Política Determinista - Accion 2
Step: 149 Número Aleatorio: 0.6434211004351358 Epsilon: 0.4337999999999962
Step: 149 x: -0.13132718  v: 0.004959857
Step: 149 Política Determinista - Accion 2
Step: 150 Número Aleatorio: 0.30840359795427197 Epsilon: 0.42999999999999616
Step: 150 Política Aleatoria - Accion 2
Step: 151 Número Aleatorio: 0.6697706916463616 Epsilon: 0.42619999999999614
Step: 151 x: -0.12534325  v: 0.0023325435
Step: 151 Política Determinista - Accion 2
Step: 152 Número Aleatorio: 0.37156918735031885 Epsilon: 0.4223999999999961
Step: 152 Política Aleatoria - Accion 1
Step: 153 Número Aleatorio: 0.3592737938964403 Epsilon

Step: 206 Número Aleatorio: 0.9680674298024724 Epsilon: 0.21719999999999498
Step: 206 x: -0.8375883  v: 0.0455042
Step: 206 Política Determinista - Accion 2
Step: 207 Número Aleatorio: 0.2764035979954491 Epsilon: 0.21339999999999498
Step: 207 x: -0.7890623  v: 0.048525997
Step: 207 Política Determinista - Accion 2
Step: 208 Número Aleatorio: 0.06323913703496775 Epsilon: 0.20959999999999498
Step: 208 Política Aleatoria - Accion 0
Step: 209 Número Aleatorio: 0.49183513961345837 Epsilon: 0.205799999999995
Step: 209 x: -0.68593824  v: 0.051810987
Step: 209 Política Determinista - Accion 2
Step: 210 Número Aleatorio: 0.9502401894049981 Epsilon: 0.201999999999995
Step: 210 x: -0.6319573  v: 0.05398097
Step: 210 Política Determinista - Accion 2
Step: 211 Número Aleatorio: 0.03620881854475011 Epsilon: 0.198199999999995
Step: 211 Política Aleatoria - Accion 1
Step: 212 Número Aleatorio: 0.17861721759817173 Epsilon: 0.194399999999995
Step: 212 Política Aleatoria - Accion 1
Step: 213 Número Aleat

In [36]:
env_wrapper_probabilistico(policy_probabilista, env,steps,0.25) # O invoca tu wrapper modificado

Step: 0 Número Aleatorio: 0.5014600941769188 Epsilon: 0.25
Step: 0 x: -0.54130685  v: 0.0
Step: 0 Política Determinista - Accion 2
Step: 1 Número Aleatorio: 0.5929279566878536 Epsilon: 0.2462
Step: 1 x: -0.5401741  v: 0.0011327482
Step: 1 Política Determinista - Accion 2
Step: 2 Número Aleatorio: 0.5186380547758342 Epsilon: 0.2424
Step: 2 x: -0.5379171  v: 0.0022570118
Step: 2 Política Determinista - Accion 2
Step: 3 Número Aleatorio: 0.8139967089504148 Epsilon: 0.2386
Step: 3 x: -0.53455275  v: 0.0033643662
Step: 3 Política Determinista - Accion 2
Step: 4 Número Aleatorio: 0.22735082541001927 Epsilon: 0.2348
Step: 4 Política Aleatoria - Accion 1
Step: 5 Número Aleatorio: 0.20629788091188095 Epsilon: 0.231
Step: 5 Política Aleatoria - Accion 0
Step: 6 Número Aleatorio: 0.24421603864892893 Epsilon: 0.2272
Step: 6 x: -0.5221005  v: 0.0035104
Step: 6 Política Determinista - Accion 2
Step: 7 Número Aleatorio: 0.797713258322568 Epsilon: 0.22340000000000002
Step: 7 x: -0.5176014  v: 0.004499

Step: 57 Número Aleatorio: 0.05629457009622574 Epsilon: 0.048600000000000115
Step: 57 x: -0.460531  v: -0.024626536
Step: 57 Política Determinista - Accion 0
Step: 58 Número Aleatorio: 0.5501164068524459 Epsilon: 0.048600000000000115
Step: 58 x: -0.48662773  v: -0.026096728
Step: 58 Política Determinista - Accion 0
Step: 59 Número Aleatorio: 0.1954913216708134 Epsilon: 0.048600000000000115
Step: 59 x: -0.5140012  v: -0.027373442
Step: 59 Política Determinista - Accion 0
Step: 60 Número Aleatorio: 0.4868927784351017 Epsilon: 0.048600000000000115
Step: 60 x: -0.5424466  v: -0.028445413
Step: 60 Política Determinista - Accion 0
Step: 61 Número Aleatorio: 0.7410341005676068 Epsilon: 0.048600000000000115
Step: 61 x: -0.5717507  v: -0.02930413
Step: 61 Política Determinista - Accion 0
Step: 62 Número Aleatorio: 0.2537310235366892 Epsilon: 0.048600000000000115
Step: 62 x: -0.60169494  v: -0.029944245
Step: 62 Política Determinista - Accion 0
Step: 63 Número Aleatorio: 0.2597314980894242 Epsil

Step: 116 Número Aleatorio: 0.6312329217085451 Epsilon: 0.048600000000000115
Step: 116 x: -0.10778871  v: 0.053320974
Step: 116 Política Determinista - Accion 2
Step: 117 Número Aleatorio: 0.4484085045406483 Epsilon: 0.048600000000000115
Step: 117 x: -0.055838168  v: 0.051950544
Step: 117 Política Determinista - Accion 2
Step: 118 Número Aleatorio: 0.18219703197122117 Epsilon: 0.048600000000000115
Step: 118 x: -0.0053526266  v: 0.05048554
Step: 118 Política Determinista - Accion 2
Step: 119 Número Aleatorio: 0.1555148403309401 Epsilon: 0.048600000000000115
Step: 119 x: 0.043633234  v: 0.04898586
Step: 119 Política Determinista - Accion 2
Step: 120 Número Aleatorio: 0.22277468788819732 Epsilon: 0.048600000000000115
Step: 120 x: 0.09114049  v: 0.04750725
Step: 120 Política Determinista - Accion 2
Step: 121 Número Aleatorio: 0.5472677108278953 Epsilon: 0.048600000000000115
Step: 121 x: 0.1372406  v: 0.046100117
Step: 121 Política Determinista - Accion 2
Step: 122 Número Aleatorio: 0.19811

## Usando velocidad del coche

Ahora queremos utilizar la velocidad actual del coche (segundo elemento de la observación retornada por el entorno) para lograr el objetivo.

In [37]:
def policy_speed(observation, step, *args):
    speed = observation[1]
    action = 0 if speed < 0 else 2# YOUR CODE HERE # solo velocidad no la posición parecido al que hemos hecho.
    
    return action
 

In [38]:
env_wrapper(policy_speed, env, steps)

Enhorabuena! Lo has logrado en 121 pasos


## Usando *nuestra* velocidad del coche (Ejercicio extra)

Es evidente que la información que tengamos sobre el estado es muy importante para establecer políticas efectivas. En este caso, el propio entorno de `gym` nos aporta la posición y la velocidad, pero hay otros datos que no los da directamente y podrían ser útiles, como la altura o la aceleración.

El tener toda la información relevante para caracterizar un estado es vital, y está estrechamente relacionado con que se cumpla la propiedad Markoviana que estudiaremos en la siguiente [lección](../../s2/s2_worksheet.ipynb).

Imagina que el entorno sólo diera la posición en cada estado, y no la velocidad. 
* ¿Sería posible, dada la información de sólo un estado, estimar la velocidad del mismo estado?
* ¿Qué pasa si, además de tener la información del estado actual, conservamos la información del estado immediatamente anterior?

Esbribe una política exitosa utilizando sólamente las posiciones del estado actual y el inmediatamente anterior.

In [39]:
steps = 250
def policy_speed_markovian(observation, step, previous_observation):
    posicion = observation[0]
    posicion_prev = previous_observation[0]
    print("Pos: %s. Prev Pos: %s." % (posicion, posicion_prev))
    action = 0 if (posicion-posicion_prev) < 0 else 2 # YOUR CODE HERE #
    return action 
#Markov solo depende del estado anterior.

In [40]:
env_wrapper(policy_speed_markovian, env, steps)

Pos: -0.46817857. Prev Pos: -0.46817857.
Pos: -0.46759233. Prev Pos: -0.46817857.
Pos: -0.46642414. Prev Pos: -0.46759233.
Pos: -0.46468267. Prev Pos: -0.46642414.
Pos: -0.46238077. Prev Pos: -0.46468267.
Pos: -0.45953542. Prev Pos: -0.46238077.
Pos: -0.45616758. Prev Pos: -0.45953542.
Pos: -0.45230207. Prev Pos: -0.45616758.
Pos: -0.4479672. Prev Pos: -0.45230207.
Pos: -0.4431947. Prev Pos: -0.4479672.
Pos: -0.43801942. Prev Pos: -0.4431947.
Pos: -0.43247893. Prev Pos: -0.43801942.
Pos: -0.4266134. Prev Pos: -0.43247893.
Pos: -0.420465. Prev Pos: -0.4266134.
Pos: -0.41407785. Prev Pos: -0.420465.
Pos: -0.40749738. Prev Pos: -0.41407785.
Pos: -0.4007702. Prev Pos: -0.40749738.
Pos: -0.39394352. Prev Pos: -0.4007702.
Pos: -0.3870649. Prev Pos: -0.39394352.
Pos: -0.3801819. Prev Pos: -0.3870649.
Pos: -0.37334168. Prev Pos: -0.3801819.
Pos: -0.36659056. Prev Pos: -0.37334168.
Pos: -0.35997397. Prev Pos: -0.36659056.
Pos: -0.35353586. Prev Pos: -0.35997397.
Pos: -0.34731862. Prev Pos: -0.3